In [1]:
import os
os.chdir("/Users/jacksonwalters/Documents/GitHub/enefit-kaggle/predict-energy-behavior-of-prosumers/")

In [2]:
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from load_data import merged_df

In [3]:
#load the training data, dropping NaN's
dataset = merged_df()

loading train data...
loading gas_prices...
loading electricity_prices...
loading forecast_weather...
merging train and gas_prices...
merging electricity_prices...
merging forecast_weather...


In [4]:
#reduce dataset to smaller size
REDUCED_DATASET_SIZE = 1_000_000
df = dataset[:REDUCED_DATASET_SIZE].copy()

In [5]:
#test-train-validation split on the data
column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]

In [6]:
#write feature_names to .csv file
import csv
feature_names = list(df.columns.values)
with open("../models/cnn_feature_names.csv", 'w') as f:
    write = csv.writer(f)
    write.writerow(feature_names)

In [24]:
#Handle the indexes and offsets as shown in the diagrams above.
#Split windows of features into (features, labels) pairs.
#Plot the content of the resulting windows.
#Efficiently generate batches of these windows from the training, evaluation, and test data, using tf.data.Datasets.
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
    self.column_indices = {name: i for i, name in enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.test))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [25]:
#define single step window
single_step_window = WindowGenerator(
    input_width=1, label_width=1, shift=1,
    label_columns=['target'])
single_step_window

#define a baseline model
class Baseline(tf.keras.Model):
  def __init__(self, label_index=None):
    super().__init__()
    self.label_index = label_index

  def call(self, inputs):
    if self.label_index is None:
      return inputs
    result = inputs[:, :, self.label_index]
    return result[:, :, tf.newaxis]

baseline = Baseline(label_index=column_indices['target'])

baseline.compile(loss=tf.keras.losses.MeanSquaredError(),
                 metrics=[tf.keras.metrics.MeanAbsoluteError()])

#run the baseline model
val_performance = {}
performance = {}
val_performance['Baseline'] = baseline.evaluate(single_step_window.val)
performance['Baseline'] = baseline.evaluate(single_step_window.test, verbose=0)

6250/6250 [==============================] - 20s 3ms/step - loss: 6482.7856 - mean_absolute_error: 2.7454


In [45]:
#define the compile and fit training run
MAX_EPOCHS = 1

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=.000_001),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [46]:
#define the convolutional window
CONV_WIDTH = 12
conv_window = WindowGenerator(
    input_width=CONV_WIDTH,
    label_width=1,
    shift=0,
    label_columns=['target'])

#convolutional neural network
#for now inserting a BatchNormalization layer instead of normalizing manually
conv_model = tf.keras.Sequential([
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(filters=32,
                           kernel_size=(CONV_WIDTH,),
                           activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1),
    tf.keras.layers.Reshape(target_shape=()),
])

In [47]:
print("Conv model on `conv_window`")
print('Input shape:', conv_window.example[0].shape)
print('Output shape:', conv_model(conv_window.example[0]).shape)

Conv model on `conv_window`
Input shape: (32, 12, 24)
Output shape: (32,)


In [42]:
conv_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_4 (Bat  (32, 12, 24)              96        
 chNormalization)                                                
                                                                 
 conv1d_4 (Conv1D)           (32, 1, 32)               9248      
                                                                 
 dense_8 (Dense)             (32, 1, 32)               1056      
                                                                 
 dense_9 (Dense)             (32, 1, 1)                33        
                                                                 
 reshape_3 (Reshape)         (32,)                     0         
                                                                 
Total params: 10433 (40.75 KB)
Trainable params: 10385 (40.57 KB)
Non-trainable params: 48 (192.00 Byte)
_______________

In [48]:
#train the model
history = compile_and_fit(conv_model, conv_window)

#evaluate performance
val_performance['Conv'] = conv_model.evaluate(conv_window.val)
performance['Conv'] = conv_model.evaluate(conv_window.test, verbose=1)

3125/3125 [==============================] - 20s 6ms/step - loss: 421606.0000 - mean_absolute_error: 174.1167


In [49]:
# save the model to disk
import pickle
cnn_model_filename = '../models/cnn_model.sav'
pickle.dump(conv_model, open(cnn_model_filename, 'wb'))

In [54]:
#save model using keras
conv_model.save('../models/cnn_model.keras')

#save model using legacy keras
conv_model.save('../models/cnn_model.h5')

In [55]:
conv_model(conv_window.example[0])

<tf.Tensor: shape=(32,), dtype=float32, numpy=
array([ 9.100809 ,  9.5893135, 10.810103 , 10.418977 , 10.649641 ,
        7.31566  ,  8.783263 ,  9.060895 ,  9.563227 ,  8.855568 ,
        7.5229936,  8.085756 ,  7.498831 , 11.097404 , 10.507688 ,
        9.65713  ,  8.133643 ,  7.882827 , 10.019332 ,  9.624953 ,
        9.987697 , 11.165125 ,  9.980533 , 11.019373 ,  8.992881 ,
        9.189162 , 10.923996 ,  7.7113495,  7.044467 ,  9.912277 ,
        9.421008 ,  9.990296 ], dtype=float32)>

In [59]:
conv_model.predict(next(iter(conv_window.val))[0])

1/1 [==============================] - 0s 10ms/step


array([7.7927623, 6.436409 , 6.5068274, 5.9486175, 5.460173 , 7.3531957,
       5.137743 , 5.2244844, 6.752815 , 6.929317 , 7.7764473, 6.024827 ,
       5.8012033, 7.5677733, 6.253757 , 7.9643264, 6.1561403, 7.960485 ,
       7.0253773, 7.8343205, 6.9852386, 6.8489156, 7.372115 , 8.234603 ,
       5.6909256, 7.098112 , 7.8986735, 6.9408593, 7.2767396, 7.7127075,
       7.123909 , 7.810176 ], dtype=float32)

In [57]:
conv_model.predict(conv_window.example[0])

1/1 [==============================] - 0s 18ms/step


array([ 9.100809 ,  9.5893135, 10.810103 , 10.418977 , 10.649641 ,
        7.31566  ,  8.783263 ,  9.060895 ,  9.563227 ,  8.855568 ,
        7.5229936,  8.085756 ,  7.498831 , 11.097404 , 10.507688 ,
        9.65713  ,  8.133643 ,  7.882827 , 10.019332 ,  9.624953 ,
        9.987697 , 11.165125 ,  9.980533 , 11.019373 ,  8.992881 ,
        9.189162 , 10.923996 ,  7.7113495,  7.044467 ,  9.912277 ,
        9.421008 ,  9.990296 ], dtype=float32)